# Import dependencies

In [ ]:
import pandas as pd
import requests
import os
import json
from typing import List
from util.webscraper import WebScraper
import numpy as np
import nltk
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

https://python.langchain.com/v0.2/docs/tutorials/rag/

# Data Collection

First, we get the data from the API. As the API is not yet published, both the API-Url and the query to get information on edition-software need to be specified in your .env file. (consult the README for more information)

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
# get api_url and query
api_url = os.environ['API_URL']
query = os.environ['QUERY']

# get data from api
api_response = requests.get(api_url + query)

Now that we got the data from the API, we can load it into a dataframe to prepare it to be used as a knowledge base for rag. 

In [ ]:
edition_software_info = json.loads(api_response.text)
edition_software_info = pd.DataFrame(edition_software_info)
edition_software_info.info()

A brief inspection allows us to formulate some initial tasks and questions for this experiment.

- **Preprocessing:** As we can see, not a single entry contains a associated concept_doi. We might consider dropping the column.
- **Impact of using short descriptions only:** Three entries are missing the in depth description. We can assume that rag won't be too useful for these entries. 
- **Impact of additional information:** Only three have a description-url. Down the road, we need to evaluate, if adding info from this source improves the performance of the rag-system.

# Preprocessing

### 1. Remove Artefacts

Both the `description` and `short_statement` columns seem to be of particular interest for the task at hand. To asses necessary preprocessing step, we'll need to take a closer look at them.

In [ ]:
descriptions = edition_software_info[["description", "short_statement"]]
with pd.option_context('display.max_colwidth', None):
    display(descriptions.head())

As we can see, the `description` column contains some formatting artefacts like `\n` and markdown syntax like `**` and `#`. Let's clean them up.
While we're at it, we can also remove double whitespaces etc.

- To do: [CollateX](http://collatex.net/) -> Links
- Markdown vielleicht sogar behalten???

In [ ]:
pattern = '\\n+'
edition_software_info["description_clean"] = edition_software_info["description"].str.replace(pattern, ' ', regex=True)

pattern = r'[*#]+|\s-+\s|]]' #\[\]()<>
edition_software_info["description_clean"] = edition_software_info["description_clean"].str.replace(pattern, ' ', regex=True)

with pd.option_context('display.max_colwidth', None):
    display(edition_software_info[["brand_name", "description", "description_clean"]].head())

### 3. Fill nan values

Before we continue preprocessing the data for later vectorization, we need to check for missing values and replace them with empty strings.

In [ ]:
edition_software_info["description_clean"].fillna("", inplace=True)

### 2. Remove Stopwords and Punctuation

For the later vectorisation of te texts, we remove both german and english stopwords.

In [ ]:
def preprocess(stopwords: List[str], text: str) -> str:
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text

In [ ]:
# get stopwords
stopwords_english = set(stopwords.words('english'))
stopwords_german = set(stopwords.words('german'))
stopwords_combined = stopwords_german.union(stopwords_english)

edition_software_info["description_preprocessed"] = edition_software_info["description_clean"].apply(lambda x: preprocess(stopwords_combined, x))
edition_software_info["description_preprocessed"].head()

# Scrape Webpages

To provide additional context-information for the retrieval process, we'll scrape all webpages referenced in the software-description.

### 1. Get urls

First, we isolate the urls from our description.

In [ ]:
pattern = r"((?:https?:\/\/|w{3}.)[\w\d%/.-]+)"

urls = edition_software_info["description"].str.extractall(pattern)
urls = urls.droplevel(1)
urls_grouped = urls.groupby(urls.index).agg((lambda x: ','.join(set(x))))
edition_software_info["urls"] = urls_grouped

with pd.option_context('display.max_colwidth', None):
    display(edition_software_info[["description_clean", "urls"]].head())

### 2. Scrape Webpages

Now we scrape the paragraphs from the webpages we found. 
The webscraper will take the list of urls associated with an entry and will save paragraphs from all webpages as a string in a column of our dataframe. 

**This might take some time**

In [ ]:
webscraper = WebScraper(tags = ["p"], exclude = ["wikipedia"])
edition_software_info["webpages_text"] = edition_software_info["urls"].apply(lambda x: webscraper.scrape(x))

In [ ]:
edition_software_info[["urls", "webpages_text"]].head()

Now that the data is collected from the webpages, we can take a look at the average length of the texts received for each entry.

In [ ]:
length = edition_software_info["webpages_text"].apply(lambda x: len(x) if not pd.isna(x) else 0)
length[length>0].describe()

Looking only at entries, that had webpages associated with them, we have an average character count of 15.139 per entry. 
The standard deviation (16,998) is quite large compared to the mean, indicating that there is a high degree of variability in character counts.

The distribution is moderately skewed towards entries with lower character counts, while some outliers with a high character counts pull the mean upwards.



### 3. Preprocessing

Now, we can preprocess the newfound text using the function we defined earlier. Again, we have to replace missing values with empty strings.

In [ ]:
edition_software_info["webpages_text_preprocessed"] = edition_software_info["webpages_text"].apply(lambda x: preprocess(stopwords_combined, x) if not pd.isna(x) else "")
edition_software_info["webpages_text_preprocessed"].head()

# Vectorization